# Calcular coordenadas y distancias a partir de direcciones

### Importar librerias



In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from geopy.geocoders import Bing
from geopy.distance import geodesic

### Importar archivos

In [23]:
direcciones = pd.read_excel("Tsp-1.xlsx")
direcciones.head()

,colonia,Calle,# de casa,Código postal,Unnamed: 4
0,U H FOVISSSTE ESTADI,LISBOA,28,44307,Guadalajara
1,INDEPENDENCIA,ROGELIO BACON,2280,44290,Guadalajara
2,BOSQUES D LA CANTERA,PRIV OPALO,17,44306,Guadalajara
3,JARDINES DE LOS POET,JOSE FERNANDEZ,3365,44820,Guadalajara
4,LOMAS DEL GALLO,FERNANDO SOLIS,1338,44760,Guadalajara


### Encontrar latitudes y longitudes

In [24]:
def obtener_coordenadas(tabla):
    coords = []
    geolocator = Bing(api_key='Ago97Rd3jwxWgFxgUWwgUrZ7S21qlE8ok8UvjoEnaXP2nl2Uh8NHqokudoSjLJzI')
    for i in range(tabla.shape[0]):
        colonia = tabla.iloc[i,0]
        calle = tabla.iloc[i,1]
        n_casa = tabla.iloc[i,2]
        codigoPostal = tabla.iloc[i,3]
        municipio = tabla.iloc[i,4]

        query = f'{calle}, {n_casa}, {colonia}, {codigoPostal}, {municipio}, México'
    
        location = geolocator.geocode(query)

        if location:
            latitude = location.latitude
            longitude = location.longitude
            #print(f'Latitude: {latitude}, Longitude: {longitude}')
            coords.append((latitude,longitude))
        else:
            coords.append((0,0))

    return coords

In [25]:
#funciones para que nos arroje las latitudes y longitudes, respectivamente
def return_latitude(x):
    return x[0]

def return_longitude(x):
    return x[1]

In [26]:
direcciones["Coordenadas"] = obtener_coordenadas(direcciones)
direcciones["Latitud"] = direcciones["Coordenadas"].apply(lambda x: return_latitude(x))
direcciones["Longitud"] = direcciones["Coordenadas"].apply(lambda x:return_longitude(x))

In [27]:
direcciones.head(10)

,colonia,Calle,# de casa,Código postal,Unnamed: 4,Coordenadas,Latitud,Longitud
0,U H FOVISSSTE ESTADI,LISBOA,28,44307,Guadalajara,"(20.7108869, -103.3308462)",20.710887,-103.330846
1,INDEPENDENCIA,ROGELIO BACON,2280,44290,Guadalajara,"(20.70184763, -103.33378321)",20.701848,-103.333783
2,BOSQUES D LA CANTERA,PRIV OPALO,17,44306,Guadalajara,"(20.600206, -103.3917429)",20.600206,-103.391743
3,JARDINES DE LOS POET,JOSE FERNANDEZ,3365,44820,Guadalajara,"(20.6484961, -103.29001645)",20.648496,-103.290016
4,LOMAS DEL GALLO,FERNANDO SOLIS,1338,44760,Guadalajara,"(20.67890219, -103.27083555)",20.678902,-103.270836
5,BATALLON SN PATRICIO,CALZADA INDEPEN,503,44300,Guadalajara,"(20.71436167, -103.3128224)",20.714362,-103.312822
6,TETLAN,GIGANTES,108,44820,Guadalajara,"(20.65942035, -103.2728094)",20.659420,-103.272809
7,MODERNA,AV ESPANA,1092,44190,Guadalajara,"(20.66398621, -103.35904694)",20.663986,-103.359047
8,RINCONADA DE HUENTITAN,JUAN CARRASCO,5234,44250,Guadalajara,"(20.7334639, -103.3219648)",20.733464,-103.321965
9,GENERAL REAL,MARCONI,17,44400,Guadalajara,"(20.6682228, -103.3242311)",20.668223,-103.324231


### Hace matriz de distancias 

In [28]:
def generar_distancias(clientes):
    matriz_distancias = []

    for i in range(clientes.shape[0]):
        coord1 = clientes.iloc[i,5]
        distancias =  []

        for j in range(clientes.shape[0]):
            if j != i:
                coord2 = clientes.iloc[j,5]
                distancia = float(geodesic(coord1, coord2).km)
                distancias.append(distancia)
            else:
                distancias.append(999999)

        matriz_distancias.append(distancias)

    distancias= pd.DataFrame(np.matrix(matriz_distancias))
    return distancias


### Exportar 10 redes con 40,100,150 y 200 nodos aleatoriamente

In [29]:
# Crear una lista para almacenar los dataframes de cada red
for i in range(10):
    # Seleccionar 40 direcciones de forma aleatoria sin reemplazo
    red40 = direcciones.sample(n=40, replace=False)
    red100 = direcciones.sample(n=100, replace=False)
    red150 = direcciones.sample(n=150, replace=False)
    red200 = direcciones.sample(n=200, replace=False)

    distancias40 = generar_distancias(red40)
    distancias100 = generar_distancias(red100)
    distancias150 = generar_distancias(red150)
    distancias200 = generar_distancias(red200)

    red40.to_csv(f'direcciones/40 nodos/direcciones{i}.csv',index=0)
    red100.to_csv(f'direcciones/100 nodos/direcciones{i}.csv',index=0)
    red150.to_csv(f'direcciones/150 nodos/direcciones{i}.csv',index=0)
    red200.to_csv(f'direcciones/200 nodos/direcciones{i}.csv',index=0)

    distancias40.to_csv(f'matrizDistancias/40 nodos/matrizDistancia{i}.csv')
    distancias100.to_csv(f'matrizDistancias/100 nodos/matrizDistancia{i}.csv')
    distancias150.to_csv(f'matrizDistancias/150 nodos/matrizDistancia{i}.csv')
    distancias200.to_csv(f'matrizDistancias/200 nodos/matrizDistancia{i}.csv')


direcciones.to_csv("direcciones_totales.csv",index=0)